In [1]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt 

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size 
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
print(Xb.shape, Yb.shape, C.shape)

torch.Size([32, 3]) torch.Size([32]) torch.Size([27, 10])


In [9]:
b1.shape

torch.Size([64])

In [10]:
# forward pass (chunked into small pieces)
emb = C[Xb]                                     # embed chars into vectors                  (32,3,10)
embcat = emb.view(emb.shape[0],-1)              # concatanate vectors                       (32,30)

# linear layer 
hprebn = embcat @ W1 + b1                       # hidden layer pre-activation,(before BN)   (32,30) @ (30,64) -> (32,64) + (64) = (32,64)

# BN layer 
bnmeani = 1/n * hprebn.sum(0,keepdim=True)     # batch norm mean                            (1,64)
bndiff = (hprebn - bnmeani)                    # difference                                 (32,64) -(1,64) -> (32,64)
bndiff2 = bndiff**2 
bnvar = 1/(n-1)* (bndiff2).sum(0,keepdim=True) # bn varience                                (1,64)
bnvar_inv = (bnvar + 1e-5)**-0.5               # bn var inverse                             (1,64)
bnraw = bndiff * bnvar_inv                     # bn values before the gain and bias         (32,64) * (1,64) -> (32,64)
hpreact = bngain * bnraw + bnbias              # BN                                         (1,64) * (32,64) + (1,64) -> (32,64)

# Non linearity 
h = torch.tanh(hpreact)                        # hidden layer                              (32,64)

# linear layer 2
logits = h @ W2 + b2                                                                       # (32,64) @ (64,27) + (27) = (32,27)

# cross entrophy loss 
logit_maxes = logits.max(1,keepdim=True).values                                            # (32,1)
norm_logits = logits - logit_maxes            # substract max for stability                # (32,27) -(32,1) = (32,27)
counts = norm_logits.exp()                    # counts                                     # (32,27)
counts_sum = counts.sum(1,keepdims=True)      # counts sum                                 # (32,1)
counts_sum_inv = counts_sum**-1               #                                              (32,1)
probs = counts * counts_sum_inv               # probabilities                              # (32,27) @ (32,1) -> (32,27)
logprobs = probs.log()                       # log probs                                  # (32,27)
loss = -logprobs[range(n),Yb].mean()         # negtive log probs                          # sum(32) = 1 

In [11]:
# pytorch backward pass 

for p in parameters:
    p.grad = None 

loss.backward()
loss

tensor(3.3384, grad_fn=<NegBackward0>)

In [12]:
    
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes,
        logits, h, hpreact, bnraw,bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,embcat, emb]:
    t.retain_grad()
